# Evaluación Modular:
# Interpretabilidad de Scoring Crediticio

## Objetivo
Desarrollar un modelo predictivo para el scoring crediticio, evaluando su rendimiento y la capacidad de interpretación de sus decisiones. Los estudiantes implementarán un modelo de clasificación con regularización y emplearán técnicas de interpretabilidad como SHAP o LIME para explicar el comportamiento del modelo.

**Datasets utilizados:**  
`Credit`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

1. **Carga y preprocesamiento de datos:**
- Se trabaja con el dataset Creit. Se realiza limpieza de datos, incluyendo manejo y eliminación de outliers y escalado.

2. **Entrenamiento y evaluación de modelos:**
- Se aplican regresión Lasso y regresión Ridge. Los hiperparámetros se optimizan con Optuna usando validación cruzada.

3. **Visualización e interpretación:**
- Se comparan las métricas y coeficientes de ambos modelos con tablas resumen, ademas de gráficos de barras de importancia de variables y visualización con SHAP.

---

# 2. Configuración del entorno

--- 

In [ ]:
import warnings
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import shap
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Silenciar logs de Optuna y warnings generales
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore")

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga y preprocesamiento de datos y entrenamiento de modelos.

- **`eliminar_outliers_filas()`** 
Elimina filas con valores extremos en variables críticas para evitar sesgos fuertes en el modelo.

- **`aplicar_winsorize_iqr()`** 
Limita valores atípicos moderados en variables numéricas mediante winsorización basada en IQR, preservando la mayoría de los datos..

- **`cargar_y_preprocesar_credit()`** 
Orquesta la carga del dataset, aplica las dos funciones anteriores y realiza un escalado de los datos para entregar un conjunto limpio y listo para modelado.

---

`Justificacion de la limpieza de datos:`

La estrategia combinada de limpieza y escalado se fundamenta en las siguientes razones:

- Eliminación de outliers extremos: Algunos valores muy alejados de la mayoría pueden distorsionar fuertemente la estimación del modelo y sesgar sus resultados. Por eso se eliminan filas con outliers críticos en variables clave, especialmente cuando estos valores son escasos pero con magnitudes muy elevadas.

- Winsorización para atípicos moderados: Valores atípicos menos extremos, que podrían tener impacto pero no justificarían eliminación total, se ajustan con winsorización basada en el rango intercuartílico (IQR). Esto permite preservar la mayoría de la información, mitigando el efecto de estos valores sin perder datos.

- Escalado de variables numéricas: Después de la limpieza, se aplica escalado estándar (o similar) para que todas las variables numéricas estén en una escala comparable. Esto es fundamental para modelos que dependen de magnitudes relativas o que incluyen regularización (como Lasso, Ridge o ElasticNet), ya que evita que variables con rangos amplios dominen el proceso de entrenamiento. Además, el escalado mejora la estabilidad numérica y la velocidad de convergencia.

- Por qué no solo escalado: El escalado por sí solo no corrige la distorsión causada por outliers extremos. Estos valores siguen estando presentes y pueden afectar negativamente la capacidad del modelo para generalizar. Por lo tanto, la limpieza previa (eliminación y winsorización) asegura que los datos escalados sean representativos y no sesgados.

---

In [ ]:
def eliminar_outliers_filas(df, columnas_objetivo, umbral=2):
    """
    Elimina filas donde los valores en las columnas objetivo superan el umbral especificado.

    Args:
        df (pd.DataFrame): DataFrame original.
        columnas_objetivo (list): Lista de nombres de columnas numéricas a evaluar.
        umbral (float): Valor límite para eliminar outliers.

    Returns:
        pd.DataFrame: DataFrame sin las filas consideradas outliers extremos.
    """
    mascara_valida = np.ones(len(df), dtype=bool)
    for columna in columnas_objetivo:
        mascara_valida &= (df[columna] < umbral)
    return df[mascara_valida].copy()

def aplicar_winsorize_iqr(df, columnas_excluidas=[]):
    """
    Aplica winsorización por IQR a las columnas numéricas del DataFrame (excepto las excluidas).

    Args:
        df (pd.DataFrame): DataFrame con datos numéricos.
        columnas_excluidas (list): Lista de columnas que no se modificarán.

    Returns:
        pd.DataFrame: DataFrame con winsorización aplicada a columnas numéricas.
    """
    df_wins = df.copy()
    columnas_numericas = df_wins.select_dtypes(include=['float64', 'int64']).columns
    columnas_a_modificar = [col for col in columnas_numericas if col not in columnas_excluidas]

    for columna in columnas_a_modificar:
        q1 = df_wins[columna].quantile(0.25)
        q3 = df_wins[columna].quantile(0.75)
        iqr = q3 - q1
        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr
        df_wins[columna] = df_wins[columna].clip(lower=limite_inferior, upper=limite_superior)

    return df_wins

def cargar_y_preprocesar_credit():
    """
    Carga y preprocesa el dataset 'credit' de OpenML para clasificación binaria.

    Pasos realizados:
    - Carga del dataset 'credit'.
    - Elimina outliers extremos por fila en columnas específicas.
    - Aplica winsorización IQR al resto de columnas numéricas.
    - Separa variables predictoras y objetivo ('SeriousDlqin2yrs').
    - Convierte la variable objetivo a tipo entero.

    Returns:
        X (pd.DataFrame): Variables predictoras preprocesadas.
        y (pd.Series): Variable objetivo binaria.
    """
    dataset = fetch_openml('credit', version=1, as_frame=True)
    df = dataset.frame

    columnas_outliers = ['NumberOfTimes90DaysLate', 'NumberOfTime60-89DaysPastDueNotWorse']
    df_filtrado = eliminar_outliers_filas(df, columnas_outliers, umbral=2)
    df_winsorizado = aplicar_winsorize_iqr(df_filtrado, columnas_excluidas=columnas_outliers)

    X = df_winsorizado.drop(columns='SeriousDlqin2yrs')
    y = df_winsorizado['SeriousDlqin2yrs'].astype(int)

    # Escalar solo columnas numéricas
    columnas_numericas = X.select_dtypes(include=['float64', 'int64']).columns
    scaler = StandardScaler()
    X[columnas_numericas] = scaler.fit_transform(X[columnas_numericas])
    
    return X, y

**Bloque 2:** Entrenamiento y optimización de modelos.

- **`entrenar_y_optimizar_modelos()`** 
Entrena y optimiza mediante optuna modelos de regresión logística con penalización Lasso, Ridge y ElasticNet.

---

`Justificacion para el uso de StratifiedKFold como reemplazo de train_test_split:`

No se realizó una división explícita de los datos en conjunto de entrenamiento y prueba (train_test_split) porque la validación cruzada estratificada ya cumple con ese propósito de forma más completa. Técnicamente, cada iteración de StratifiedKFold realiza una partición del conjunto en una combinación de datos de entrenamiento y validación, de modo que cada observación del dataset es utilizada tanto para entrenar como para validar el modelo, pero nunca en la misma iteración.

Esto proporciona una estimación del desempeño general del modelo que es menos sensible al azar que una sola división, reduciendo la varianza de la evaluación. Además:

- Al no reservar explícitamente un conjunto de prueba, se aprovecha el 100% de los datos para la validación cruzada, maximizando la cantidad de datos usada para entrenar en cada pliegue.
- En contextos donde la selección de modelos y la evaluación se realiza únicamente mediante validación cruzada, no es imprescindible separar un conjunto de test. Esta práctica es común y aceptada en tareas de exploración, análisis comparativo de modelos y optimización de hiperparámetros.

Sin embargo, si el objetivo fuera reportar una métrica final para generalización fuera de muestra, especialmente en producción o publicación de resultados, se recomienda una división adicional (hold-out) como train/valid/test para evitar el "data leakage" indirecto por sobreajuste al proceso de validación cruzada.

---

`Justificacion para la elección de modelos:`

Se optó por entrenar un modelo de Regresión Logística, ya que este permite aplicar de forma directa técnicas de regularización L1 (Lasso) y L2 (Ridge) mediante el parámetro penalty del estimador LogisticRegression en Scikit-learn.

Esto permite mejorar la capacidad de generalización del modelo al reducir el sobreajuste (overfitting), especialmente en presencia de muchas variables o correlaciones. La regularización L1, en particular, también permite realizar una selección automática de variables, ya que puede forzar algunos coeficientes a ser exactamente cero.

En cambio, Random Forest no permite aplicar regularización L1 o L2 de manera directa, dado que su funcionamiento se basa en la construcción de árboles de decisión, los cuales no involucran coeficientes que puedan ser penalizados en forma lineal. Por esta razón, la Regresión Logística fue la opción adecuada para cumplir con ambos requerimientos: entrenamiento del modelo y aplicación de técnicas de regularización.

---

In [ ]:
def entrenar_y_optimizar_modelos(X, y, n_folds=5, n_trials=20):
    """
    Entrena y optimiza modelos de regresión logística con Lasso, Ridge y Elastic Net usando Optuna.

    Args:
        X (pd.DataFrame): Variables predictoras.
        y (pd.Series): Variable objetivo.
        n_folds (int): Número de particiones para validación cruzada.
        n_trials (int): Número de iteraciones para Optuna.

    Returns:
        pd.DataFrame: Tabla con métricas de evaluación para cada modelo.
        dict: Diccionario con los modelos optimizados.
    """
    def objetivo_optuna(trial, X, y, tipo_regularizacion):
        """
        Función objetivo para la optimización con Optuna.

        Args:
            trial (optuna.Trial): Objeto de prueba de Optuna.
            X (pd.DataFrame): Variables predictoras.
            y (pd.Series): Variable objetivo.
            tipo_regularizacion (str): 'l1', 'l2' o 'elasticnet'.

        Returns:
            float: AUC promedio de validación cruzada.
        """
        C = trial.suggest_float('C', 1e-4, 1e4, log=True)
        if tipo_regularizacion == 'elasticnet':
            l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)
            clasificador = LogisticRegression(
                penalty='elasticnet', 
                C=C, 
                solver='saga',
                l1_ratio=l1_ratio,
                max_iter=5000,
                random_state=42
            )
        else:
            clasificador = LogisticRegression(
                penalty=tipo_regularizacion, 
                C=C, 
                solver='saga', 
                max_iter=5000,
                random_state=42
            )

        modelo = Pipeline([
            ('escalador', StandardScaler()),
            ('clasificador', clasificador)
        ])

        return cross_val_score(modelo, X, y, cv=5, scoring='roc_auc').mean()

    resultados = []
    modelos = {}

    for nombre, penalizacion in [('Lasso', 'l1'), ('Ridge', 'l2'), ('ElasticNet', 'elasticnet')]:
        estudio = optuna.create_study(direction='maximize')
        estudio.optimize(lambda trial: objetivo_optuna(trial, X, y, penalizacion), n_trials=n_trials)

        mejores_params = estudio.best_params
        mejor_C = mejores_params['C']
        l1_ratio = mejores_params.get('l1_ratio', None)

        print(f"Mejor C para {nombre}: {mejor_C:.5f}", end='')
        if l1_ratio is not None:
            print(f", l1_ratio={l1_ratio:.3f}", end='')
        print(f" con AUC={estudio.best_value:.4f}")

        if penalizacion == 'elasticnet':
            clasificador = LogisticRegression(
                penalty='elasticnet',
                C=mejor_C,
                l1_ratio=l1_ratio,
                solver='saga',
                max_iter=5000,
                random_state=42
            )
        else:
            clasificador = LogisticRegression(
                penalty=penalizacion,
                C=mejor_C,
                solver='saga',
                max_iter=5000,
                random_state=42
            )

        modelo_final = Pipeline([
            ('escalador', StandardScaler()),
            ('clasificador', clasificador)
        ])
        modelo_final.fit(X, y)

        kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
        y_pred = cross_val_predict(modelo_final, X, y, cv=kf, method='predict')
        y_prob = cross_val_predict(modelo_final, X, y, cv=kf, method='predict_proba')[:, 1]

        resultados.append({
            'modelo': nombre,
            'C': mejor_C,
            'l1_ratio': l1_ratio if l1_ratio is not None else '-',
            'accuracy': accuracy_score(y, y_pred),
            'precision': precision_score(y, y_pred),
            'recall': recall_score(y, y_pred),
            'f1_score': f1_score(y, y_pred),
            'auc': roc_auc_score(y, y_prob)
        })

        modelos[nombre] = modelo_final

    return pd.DataFrame(resultados), modelos

**Bloque 3:** Visuañización mediante tablas y gráficos.

- **`mostrar_tabla_metricas()`** 
Permite comparar directamente las métricas de rendimiento de los modelos entrenados, resaltando con un gradiente visual qué modelo sobresale en cada métrica clave.

- **`mostrar_tabla_coeficientes()`** 
Entrega una tabla ordenada con los coeficientes de Lasso, Ridge y ElasticNet, permitiendo observar de manera clara qué variables tienen mayor peso y cómo varía su influencia según el tipo de regularización..

- **`graficar_coeficientes_modelos()`** 
Representa gráficamente los 10 coeficientes más importantes por modelo, diferenciando la dirección (positiva o negativa) del efecto, lo que facilita la comprensión para usuarios no técnicos.

- **`graficar_shap_modelos()`** 
Utiliza gráficos SHAP para explicar el impacto individual de cada variable en las predicciones, aportando transparencia y robustez interpretativa al modelo.

---

`Justificacion para usar estas visualizaciones:`

Para complementar la evaluación cuantitativa, se incorporaron visualizaciones y tablas comparativas que permiten interpretar y comunicar mejor los resultados obtenidos. La función mostrar_tabla_metricas resume el desempeño de los modelos en distintas métricas, facilitando su comparación directa. Por otro lado, las funciones mostrar_tabla_coeficientes y graficar_coeficientes_modelos permiten identificar las variables más influyentes en cada modelo, destacando sus efectos positivos o negativos. Finalmente, se utiliza SHAP (graficar_shap_modelos) como herramienta de interpretabilidad de caja negra, proporcionando una visualización robusta y detallada del impacto de cada variable en las predicciones del modelo. Este enfoque integrado permite no solo evaluar el rendimiento, sino también comprender la lógica detrás de las decisiones de los modelos.

---

In [ ]:
def mostrar_tabla_metricas(resultados):
    """
    Muestra la tabla de métricas de evaluación para cada modelo.

    Args:
        resultados (pd.DataFrame): Resultados con métricas por modelo.
    """
    print("\nTabla comparativa de métricas:")
    display(resultados.style.background_gradient(cmap='Blues', axis=1))

def mostrar_tabla_coeficientes(modelos, X):
    """
    Muestra tabla comparativa de coeficientes entre modelos, ordenada por magnitud.

    Args:
        modelos (dict): Diccionario con modelos entrenados.
        X (pd.DataFrame): Variables predictoras.
    """
    coeficientes = {}
    for nombre, pipeline in modelos.items():
        coef = pipeline.named_steps['clasificador'].coef_.flatten()
        coeficientes[nombre] = coef

    df_coef = pd.DataFrame(coeficientes, index=X.columns)
    df_coef['max_magnitud'] = df_coef.abs().max(axis=1)
    df_coef = df_coef.sort_values(by='max_magnitud', ascending=False).drop(columns='max_magnitud')

    print("\nTabla comparativa de coeficientes (ordenada por magnitud):")
    display(df_coef.style.background_gradient(cmap='coolwarm', subset=list(modelos.keys())))

def graficar_coeficientes_modelos(modelos, X):
    """
    Grafica los 10 coeficientes más importantes por modelo (positivo o negativo).

    Args:
        modelos (dict): Modelos entrenados.
        X (pd.DataFrame): Variables predictoras (sin escalar).
    """
    n_modelos = len(modelos)
    fig, ejes = plt.subplots(1, n_modelos, figsize=(7 * n_modelos, 6))
    if n_modelos == 1:
        ejes = [ejes]  # Para evitar error si hay un solo modelo

    fig.suptitle("Top 10 variables importantes por modelo (Coeficientes)", fontsize=16)

    for i, (nombre, pipeline) in enumerate(modelos.items()):
        coeficientes = pipeline.named_steps['clasificador'].coef_.flatten()
        importantes = pd.Series(coeficientes, index=X.columns)
        importantes = importantes.reindex(importantes.abs().sort_values(ascending=False).head(10).index)

        colores = ['green' if val > 0 else 'red' for val in importantes.values]
        sns.barplot(x=importantes.values, y=importantes.index, ax=ejes[i], palette=colores)
        ejes[i].set_title(nombre)
        ejes[i].set_xlabel('Coeficiente')

        for idx, val in enumerate(importantes.values):
            ejes[i].text(val, idx, f'{val:.2f}', va='center', ha='left' if val > 0 else 'right', color='black')

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

def graficar_shap_modelos(modelos, X):
    """
    Genera gráficos SHAP summary plot para cada modelo.

    Args:
        modelos (dict): Modelos entrenados.
        X (pd.DataFrame): Variables predictoras (sin escalar).
    """
    for nombre, pipeline in modelos.items():
        print(f"\nGráfico SHAP para {nombre}:")
        X_escalado = pipeline.named_steps['escalador'].transform(X)
        explicador = shap.Explainer(pipeline.named_steps['clasificador'], X_escalado, feature_names=X.columns)
        valores_shap = explicador(X_escalado)
        shap.summary_plot(valores_shap, X_escalado, feature_names=X.columns, show=True)


**Bloque 4:** Función de ejecución del código.

- **`main()`** 
Ejecuta todo el flujo: carga datos, limpieza, optimización, entrenamiento y evaluación los modelos Lasso y Ridge.

In [ ]:
def main():
    X, y = cargar_y_preprocesar_credit()
    resultados, modelos_entrenados = entrenar_y_optimizar_modelos(X, y)
    mostrar_tabla_metricas(resultados)
    mostrar_tabla_coeficientes(modelos_entrenados, X)
    graficar_coeficientes_modelos(modelos_entrenados, X)
    graficar_shap_modelos(modelos_entrenados, X)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# Análisis de los resultados, reflexiones y conclusión

Los tres modelos de regresión logística penalizada —**Lasso**, **Ridge** y **ElasticNet**— han mostrado un desempeño **altamente consistente** en esta ejecución, especialmente en términos de capacidad predictiva medida a través del **AUC**. Este comportamiento es esperable, ya que todos comparten la misma estructura base y se diferencian únicamente en la forma en que aplican la penalización sobre los coeficientes.

A pesar de pequeñas variaciones numéricas, las métricas de rendimiento (`accuracy`, `precision`, `recall`, `f1-score`) son prácticamente equivalentes, lo cual refuerza la idea de que el modelo es **robusto frente al tipo específico de regularización**, al menos en este conjunto de datos. Es importante recordar que estos resultados pueden **variar ligeramente entre ejecuciones**, tanto por la aleatoriedad inherente de la validación cruzada como por el proceso de optimización con Optuna.

---

## Variables más influyentes

Independientemente del tipo de regularización, los modelos coinciden en identificar un **conjunto muy claro de variables clave**, cuya magnitud de coeficiente es consistentemente alta:

- **RevolvingUtilizationOfUnsecuredLines**: Indicador de utilización del crédito disponible. Su fuerte impacto positivo sugiere que altos niveles de utilización están asociados a mayor riesgo.
- **Número de pagos atrasados en distintas franjas (30-59, 60-89, 90+ días)**: Reflejan directamente el historial de morosidad, y naturalmente se asocian con un mayor riesgo de incumplimiento.
- **Edad**: Con coeficiente negativo, sugiere que personas de mayor edad tienen menor probabilidad de incumplir, posiblemente por mayor estabilidad financiera.
- **Ingresos mensuales**: También con peso negativo, indicando que a mayor ingreso, menor es el riesgo de default.

Estas variables son **coherentes con el dominio del problema** y aportan interpretabilidad a las decisiones del modelo.

---

## ¿Deberíamos eliminar variables poco importantes?

El análisis numérico y visual de los coeficientes revela que algunas variables tienen **magnitudes muy pequeñas**, típicamente inferiores a ±0.1. Sin embargo, decidir si deben eliminarse requiere matizar, ya que eliminar variables con coeficientes pequeños en modelos como Lasso o Ridge puede simplificar el modelo, hacerlo más rápido y reducir el ruido si esas variables realmente no aportan información útil. Sin embargo, esta decisión tiene riesgos: algunas variables con coeficientes bajos pueden aportar valor en combinación con otras (redundancia informativa), y en modelos regularizados no siempre un coeficiente pequeño implica irrelevancia total. Además, usar un umbral arbitrario para eliminar variables podría deteriorar el rendimiento del modelo si se eliminan variables con efectos marginales pero consistentes.

En contextos como el **scoring crediticio**, donde la **explicabilidad y la trazabilidad** son esenciales, podría tener sentido **mantener esas variables si no afectan negativamente la interpretación**. Alternativamente, si el objetivo es simplificar al máximo sin sacrificar desempeño, puede considerarse eliminar aquellas con coeficientes muy bajos **después de validar su impacto mediante un experimento controlado**.

---

## Conclusión

Los modelos penalizados ofrecen una excelente combinación de **rendimiento, interpretabilidad y control de complejidad**. Aunque cada técnica (Lasso, Ridge, ElasticNet) tiene matices distintos en cómo manejan los coeficientes, en esta ejecución han convergido en una solución muy similar. Las variables más influyentes tienen un claro sentido desde la perspectiva del negocio crediticio, y las menos influyentes pueden evaluarse cuidadosamente antes de decidir eliminarlas.

El uso complementario de herramientas como **SHAP** fortalece aún más la transparencia del modelo, ofreciendo explicaciones individualizadas que son clave para una implementación responsable y ética en entornos financieros.

---